# Model serialization

## Import model

In [ ]:
import json

with open("Lc2ppiK.json") as stream:
    model_def = json.load(stream)

In [ ]:
from IPython.display import JSON

JSON(model_def)

In [ ]:
kinematics_def = model_def["distributions"][0]["decay_description"]["kinematics"]
kinematics_def

In [ ]:
initial_state_def = kinematics_def["initial_state"]
final_state_def = kinematics_def["final_state"]